In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math

#import xlrd
import openpyxl
import pickle

# import torch

from joblib import dump,load
from sklearn import utils,preprocessing
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso,LassoCV
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor

from tensorflow.keras import models
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

import datetime
import time

KeyboardInterrupt: 

# Model label

In [ ]:
global label
# label="PRI"
# label="FUC"
# label="RFE"
label="GP"

# Initial parameter setting

In [ ]:
global N
global n
global test_size     # 回归运行轮次和测试集大小
N=100    # runing time for prediction
n=10    # runing time for 10K-CV
test_size=0.20    # the ratio of testing set

# read data

In [ ]:
# data = pd.read_excel('list-PRI.xlsx',header=0)    # Primary features
# data = pd.read_excel('list-FUC.xlsx',header=0)    # Primary features using statistical functions
# data = pd.read_excel('list-RFE.xlsx',header=0)    # Primary features using RFE
data = pd.read_excel('list-GP.xlsx',header=0)    # Primary features using Gplearn
data = data.iloc[:,1:]
data

In [ ]:
x_data = np.array(data.iloc[:,1:])    # features
y_data = np.array(data.iloc[:,0])    # ΔGH

# The number of Sample

In [ ]:
global size
size = x_data.shape[1]

# Defined function

In [ ]:
## gerante ANNs model
def anns_model(optimizer):
    # create model
    model = Sequential()
    model.add(Dense(units=int(size), input_dim=int(size), kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='linear',kernel_initializer='normal'))
    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

In [ ]:
##  Model regression
def fitRegression(inputData,outputData,test_size,N,regr):
    r2_train = []    # training R2
    r2_test = []    # testing R2
    rmse_train = []    # training rmse
    rmse_test = []    # testing rmse
    mae_train = []
    mae_test = []
    best_r2 = []
    best_rmse = []
    for i in range(N):
        print('---------round: %d---------'%(i+1))
        X_train,X_test,y_train,y_test = train_test_split(inputData,outputData,test_size=test_size,random_state=None)    # Randomly split data                                                                                                                # 该方法对dataframe也适用
        score_train = r2_score(y_train,regr.predict(pd.DataFrame(X_train)))
        error_train = np.sqrt(sum((y_train-regr.predict(pd.DataFrame(X_train)))**2)/max(y_train.shape))   
        mae0_train = sum(abs(y_train-regr.predict(pd.DataFrame(X_train))))/max(y_train.shape)
        r2_train.append(score_train)   
        rmse_train.append(error_train)
        mae_train.append(mae0_train)
        print('r2_train:   %.3f'%score_train)
        print('RMSE_train: %.3f'%error_train)
        print('MAE_train: %.3f'%mae0_train)
        
        score_test = r2_score(y_test,regr.predict(pd.DataFrame(X_test)))
        error_test = np.sqrt(sum((y_test-regr.predict(pd.DataFrame(X_test)))**2)/max(y_test.shape))
        mae0_test = sum(abs(y_test-regr.predict(pd.DataFrame(X_test))))/max(y_test.shape)
        r2_test.append(score_test)   
        rmse_test.append(error_test)
        mae_test.append(mae0_test)
        print('r2_test:   %.3f'%score_test)
        print('RMSE_test: %.3f'%error_test)
        print('MAE_test: %.3f'%mae0_test)
        if i == 0:
            max_score = score_test           #  At run 0, record R2 and the model at this point
            worst_mae = mae_test
            y_train_best = y_train
            y_test_best = y_test
            y_train_predict_best = regr.predict(pd.DataFrame(X_train))
            y_test_predict_best = regr.predict(pd.DataFrame(X_test))
            best_r2 = score_test
            best_rmse = error_test
        if i>0:
            if score_test>max_score:
                max_score = score_test       #  Update the maximum R2 value and save the model and the data
                y_train_best = y_train
                y_test_best = y_test
                y_train_predict_best = regr.predict(pd.DataFrame(X_train))
                y_test_predict_best = regr.predict(pd.DataFrame(X_test))
                best_r2 = score_test
                best_rmse = error_test
            if mae_test>worst_mae:
                worst_mae = mae_test
        print('--------------------------') 
    return r2_train,rmse_train,mae_train,r2_test,rmse_test,mae_test,best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,worst_mae

In [ ]:
##  Hyperparameter optimization
def optHyperparameter(inputData,outputData,test_size,n,search):
    r2 = []
    for i in range(n):
        X_train,X_test,y_train,y_test = train_test_split(inputData,outputData,test_size=test_size,random_state=None)    
        search.fit(X_train,y_train)
        regr = search.best_estimator_
        r2 = r2_score(y_train,regr.predict(X_train))
        if i == 0:
            max_score = r2          
            best_regr = regr    
        if i>0:
            if r2>max_score:
                max_score = r2       
                best_regr = regr  
    return max_score,best_regr

In [ ]:
##  GridCv generation
def generateSearch(model,method,param_grid,n_jobs,cv):    
    if model=='svr':
        svr = SVR()
        if method=='grid_search':
            search = GridSearchCV(svr,param_grid=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
            return search
        elif method=='randomized_search':
            search = RandomizedSearchCV(svr,param_distributions=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
            return search     
    elif model=='krr':
        krr = KernelRidge()
        if method=='grid_search':
            search = GridSearchCV(krr,param_grid=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
            return search
        elif method=='randomized_search':
            search = RandomizedSearchCV(krr,param_distributions=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
            return search
    elif model=='rfr':
        rfr = RandomForestRegressor()
        if method=='grid_search':
            search = GridSearchCV(rfr,param_grid=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
            return search
        elif method=='randomized_search':
            search = RandomizedSearchCV(rfr,param_distributions=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
            return search
    elif model=='xgbr':
        xgbr = XGBRegressor()
        if method=='grid_search':
            search = GridSearchCV(xgbr,param_grid=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
            return search
        elif method=='randomized_search':
            search = RandomizedSearchCV(xgbr,param_distributions=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
            return search
    elif model=='lasso':
        lasso = Lasso()
        if method=='grid_search':
            search = GridSearchCV(lasso,param_grid=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
            return search
        elif method=='randomized_search':
            search = RandomizedSearchCV(lasso,param_distributions=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
    elif model=='anns':
        anns = KerasRegressor(anns_model)
        if method=='grid_search':
            search = GridSearchCV(estimator = anns,param_grid=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
            return search
        elif method=='randomized_search':
            search = RandomizedSearchCV(estimator = anns,param_distributions=param_grid,n_jobs=n_jobs,cv=cv,scoring = 'r2')
            return search

In [ ]:
##  Draw bar
def plotBar(r2_train,rmse_train,r2_test,rmse_test,N,model,label,saving):
    mpl.rcParams['font.sans-serif']=['SimHei'] 
    mpl.rcParams['axes.unicode_minus']=False 
    plt.figure(figsize=(6,6))
    rmse_r2 = [rmse_train, rmse_test, r2_train, r2_test]
    mean_rmse_r2 = np.mean(rmse_r2,1) #average
    std_err = np.std(rmse_r2,axis=1,ddof=1)/np.sqrt(N) # error bar
    error_attri = dict(elinewidth=2, ecolor="black",capsize=3)
    x = np.arange(len(rmse_r2))
    rects = plt.bar(x,mean_rmse_r2,color="c",width=0.6,align="center",
            yerr=std_err,error_kw=error_attri,
            tick_label=["Training rmse","Testing rmse","Training R2","Testing R2"])
    plt.tick_params(labelsize=8)
    for x,mean_rmse_r2,std_err in zip(x,mean_rmse_r2,std_err):
        text_str = '%.3f'%mean_rmse_r2+"±"+'%.3f'%std_err
        plt.text(x,mean_rmse_r2+std_err*1.1,text_str,ha='center',va='bottom',fontsize=10)
    plt.ylim([0,1.2])
    plt.title("Random "+str(N)+" times"+model+"prediction",fontsize=10)
    if saving == 'saving':
        plt.savefig(model+'_plotbar_'+label+'.png')
    plt.show()

In [ ]:
##  Draw scatter
def plotScatter(best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,model,label,saving):
    mpl.rcParams['font.sans-serif']=['SimHei']
    mpl.rcParams['axes.unicode_minus']=False 
    R2_train = r2_score(y_train_best,y_train_predict_best)
    rmse_train = np.sqrt(sum((y_train_best-y_train_predict_best)**2)/max(y_train_best.shape))
    plt.figure(figsize=(6,6))
    plt.scatter(y_train_predict_best,y_train_best,
            c='limegreen',
            edgecolor='white',
            marker='s',
            s=50,
            alpha=0.9,
            label='Training Data')
    plt.scatter(y_test_predict_best,y_test_best,
            c='steelblue',
            edgecolor='white',
            marker='o',
            s=50,
            alpha=0.9,
            label='Test Data')
# plt.plot(line_x,line_y,linestyle='--',color='black',alpha=0.7)
    plt.xlabel("Predicted Delta_G",fontsize=10)
    plt.ylabel("Actual Delta_G",fontsize=10)
    plt.tick_params(labelsize=10)
    plt.legend(loc='upper left',fontsize=10)
    plt.title(model+'Training RMSE:%.2f,R2:%.2f Testing RMSE:%.2f,R2:%.2f'%(rmse_train,R2_train,best_rmse,best_r2),fontsize=10)
    if saving == 'saving':
        plt.savefig(model+'_plotscatter'+label+'.png')
    plt.show()
    Y_predict = np.concatenate((y_train_predict_best,y_test_predict_best),axis=0)
    Y_act = np.concatenate((y_train_best,y_test_best),axis=0)
    pd.DataFrame(Y_predict).to_csv('A.csv',sep=',')
    pd.DataFrame(Y_act).to_csv('B.csv',sep=',')
    Y_total = pd.DataFrame(np.array([Y_predict, Y_act]).T)
    Y_total.columns = ["Predicted GH", "Actual GH"]
    Y_total.to_csv("Best\\"+model+'_best_prediction_'+label+'.csv',sep=',')

# The beginning of time

In [ ]:
start_time = datetime.datetime.now()
time.sleep(5)
print(start_time)

# SVR

In [ ]:
def generateSvrParamGrid(kernel,low1,high1,num1,base1,degree,gamma,low2,high2,num2,base2,epsilon):      # 
    if isinstance(gamma,int) or isinstance(gamma,float):
        param_grid = {
            'kernel':kernel,
            'C':np.logspace(low1,high1,num1,base=base1),
            'degree':degree,
            'gamma':np.logspace(low2,high2,num2,base=base2),
            'epsilon':epsilon
        }
        return param_grid
    elif isinstance(gamma,list):
        param_grid = {
            'kernel':kernel,
            'C':np.logspace(low1,high1,num1,base=base1),
            'degree':degree,
            'gamma':gamma,
            'epsilon':epsilon
        }
        return param_grid

In [ ]:
param = generateSvrParamGrid(['linear','poly','rbf'],-3,3,20,2,[3],['scale'],0,0,0,0,[0.001])
param

In [ ]:
search = generateSearch('svr','grid_search',param,-1,10)
search

In [ ]:
max_score,best_regr = optHyperparameter(x_data,y_data,test_size,n,search)

In [ ]:
max_score

In [ ]:
best_regr

In [ ]:
r2_train,rmse_train,mae_train,r2_test,rmse_test,mae_test,best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,svr_worst_mae= \
fitRegression(x_data,y_data,test_size,N,best_regr)

In [ ]:
plotBar(r2_train,rmse_train,r2_test,rmse_test,N,'SVR',label,'saving')

In [ ]:
plotScatter(best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,'SVR',label,'saving')

In [ ]:
best_svr = best_regr

In [ ]:
dump(best_svr, "Models\\svr_"+label+".joblib")    # Save to hard disk

In [ ]:
rmse_r2 = [rmse_train, rmse_test, r2_train, r2_test]
mean_rmse_r2 = np.mean(rmse_r2,1) 
std_err = np.std(rmse_r2,axis=1,ddof=1)/np.sqrt(N) 
svr_predicted_result = [mean_rmse_r2,std_err]
svr_predicted_result = pd.DataFrame(svr_predicted_result)
svr_predicted_result.columns = ["Training rmse", "Testing rmse", "Training R2", "Testing R2"]
svr_predicted_result.index = ["Mean", "std"]
svr_predicted_result.to_csv("Excels\\"+"SVR_"+label+".csv",sep=',')
svr_mae = [mae_train, mae_test]
svr_mae = pd.DataFrame(svr_mae)
svr_mae.to_csv("Excels\\"+"SVR_MAE.csv",sep=',')

In [ ]:
end_time_svr = datetime.datetime.now()
delta_svr = end_time_svr - start_time
print(end_time_svr)
print(delta_svr)

# KRR

In [ ]:
def generateKrrParamGrid(low1,high1,num1,base1,kernel,low2,high2,num2):      # 
    param_grid = {
        'alpha':np.logspace(low1,high1,num1,base=base1),
        'kernel':kernel,
        'degree':np.arange(low2,high2,num2)
    }
    return param_grid

In [ ]:
param = generateKrrParamGrid(-6,6,13,10,['linear','poly','rbf'],1,7,0.5)
param

In [ ]:
search = generateSearch('krr','grid_search',param,-1,10)
search

In [ ]:
max_score,best_regr = optHyperparameter(x_data,y_data,test_size,n,search)

In [ ]:
max_score

In [ ]:
best_regr

In [ ]:
r2_train,rmse_train,mae_train,r2_test,rmse_test,mae_test,best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,krr_worst_mae= \
fitRegression(x_data,y_data,test_size,N,best_regr)

In [ ]:
plotBar(r2_train,rmse_train,r2_test,rmse_test,N,'KRR',label,'saving')

In [ ]:
plotScatter(best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,'KRR',label,'saving')

In [ ]:
best_krr = best_regr

In [ ]:
dump(best_krr, "Models\\krr_"+label+".joblib")    # 存储至硬盘

In [ ]:
rmse_r2 = [rmse_train, rmse_test, r2_train, r2_test]
mean_rmse_r2 = np.mean(rmse_r2,1) 
std_err = np.std(rmse_r2,axis=1,ddof=1)/np.sqrt(N) 
krr_predicted_result = [mean_rmse_r2,std_err]
krr_predicted_result = pd.DataFrame(krr_predicted_result)
krr_predicted_result.columns = ["Training rmse", "Testing rmse", "Training R2", "Testing R2"]
krr_predicted_result.index = ["Mean", "std"]
krr_predicted_result.to_csv("Excels\\"+"KRR_"+label+".csv",sep=',')
krr_mae = [mae_train, mae_test]
krr_mae = pd.DataFrame(krr_mae)
krr_mae.to_csv("Excels\\"+"KRR_MAE.csv",sep=',')

In [ ]:
end_time_krr = datetime.datetime.now()
delta_krr = end_time_krr - end_time_svr
print(end_time_krr)
print(delta_krr)

# RFR

In [ ]:
def generateRfrParamGrid(low1,high1,num1,low2,high2,num2):      # 
    param_grid = {
        'n_estimators':np.arange(int(low1),int(high1),int(num1)),
        'max_depth':np.arange(int(low2),int(high2),int(num2))
    }
    return param_grid

In [ ]:
# param = generateRfrParamGrid(100,1000,10,1,max(math.floor(x_data.shape[1]/3),1),1)
param = generateRfrParamGrid(300,500,10,3,7,1)
param 

In [ ]:
search = generateSearch('rfr','grid_search',param,-1,10)
search

In [ ]:
max_score,best_regr = optHyperparameter(x_data,y_data,test_size,n,search)

In [ ]:
max_score

In [ ]:
best_regr

In [ ]:
r2_train,rmse_train,mae_train,r2_test,rmse_test,mae_test,best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,rfr_worst_mae= \
fitRegression(x_data,y_data,test_size,N,best_regr)

In [ ]:
plotBar(r2_train,rmse_train,r2_test,rmse_test,N,'RFR',label,'saving')

In [ ]:
plotScatter(best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,'RFR',label,'saving')

In [ ]:
best_rfr = best_regr

In [ ]:
dump(best_rfr, "Models\\rfr_"+label+".joblib")    

In [ ]:
rmse_r2 = [rmse_train, rmse_test, r2_train, r2_test]
mean_rmse_r2 = np.mean(rmse_r2,1) 
std_err = np.std(rmse_r2,axis=1,ddof=1)/np.sqrt(N) 
rfr_predicted_result = [mean_rmse_r2,std_err]
rfr_predicted_result = pd.DataFrame(rfr_predicted_result)
rfr_predicted_result.columns = ["Training rmse", "Testing rmse", "Training R2", "Testing R2"]
rfr_predicted_result.index = ["Mean", "std"]
rfr_predicted_result.to_csv("Excels\\"+"RFR_"+label+".csv",sep=',')
rfr_mae = [mae_train, mae_test]
rfr_mae = pd.DataFrame(rfr_mae)
rfr_mae.to_csv("Excels\\"+"RFR_MAE.csv",sep=',')

In [ ]:
end_time_rfr = datetime.datetime.now()
delta_rfr = end_time_rfr - end_time_krr
print(end_time_rfr)
print(delta_rfr)

# XGBR

In [ ]:
def generateXgbrParamGrid(low1,high1,num1,low2,high2,num2,low3,high3,num3,rating,bytree,subsample,reg_alpha,reg_lambda,scale_pos_weight,gamma,job):      # 
    param_grid = {
        'n_estimators':np.arange(int(low1),int(high1),int(num1)),
        'max_depth':np.arange(int(low2),int(high2),int(num2)),
        'min_child_weight':np.arange(int(low3),int(high3),int(num3)),
        'learning_rate':[rating],
        'colsample_bytree':[bytree],
        'subsample':[subsample],
        'reg_alpha':[reg_alpha],
        'reg_lambda':[reg_lambda],
        'scale_pos_weight':[scale_pos_weight],
        'gamma':[gamma],
        'n_jobs':[job]
    }
    return param_grid

In [ ]:
param = generateXgbrParamGrid(50,100,20, 10,15,1, 5,11,1, 0.2, 1, 0.41000000000000003, 0.2, 0.79, 1, 0, 8)
param

In [ ]:
search = generateSearch('xgbr','grid_search',param,-1,10)
search

In [ ]:
max_score,best_regr = optHyperparameter(x_data,y_data,test_size,n,search)

In [ ]:
max_score

In [ ]:
best_regr

In [ ]:
r2_train,rmse_train,mae_train,r2_test,rmse_test,mae_test,best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,xgbr_worst_mae= \
fitRegression(x_data,y_data,test_size,N,best_regr)

In [ ]:
plotBar(r2_train,rmse_train,r2_test,rmse_test,N,'XGBR',label,'saving')

In [ ]:
plotScatter(best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,'XGBR',label,'saving')

In [ ]:
best_xgbr = best_regr

In [ ]:
dump(best_xgbr, "Models\\xgbr_"+label+".joblib")    # 存储至硬盘

In [ ]:
rmse_r2 = [rmse_train, rmse_test, r2_train, r2_test]
mean_rmse_r2 = np.mean(rmse_r2,1) 
std_err = np.std(rmse_r2,axis=1,ddof=1)/np.sqrt(N) 
xgbr_predicted_result = [mean_rmse_r2,std_err]
xgbr_predicted_result = pd.DataFrame(xgbr_predicted_result)
xgbr_predicted_result.columns = ["Training rmse", "Testing rmse", "Training R2", "Testing R2"]
xgbr_predicted_result.index = ["Mean", "std"]
xgbr_predicted_result.to_csv("Excels\\"+"XGBR_"+label+".csv",sep=',')
xgbr_mae = [mae_train, mae_test]
xgbr_mae = pd.DataFrame(xgbr_mae)
xgbr_mae.to_csv("Excels\\"+"XGBR_MAE.csv",sep=',')

In [ ]:
end_time_xgbr = datetime.datetime.now()
delta_xgbr = end_time_xgbr - end_time_rfr
print(end_time_xgbr)
print(delta_xgbr)

# LASSO

In [ ]:
def generateLassoParamGrid(way,low,high,num,base,norm,max_iter):      
    if way=='logspace':
        param_grid = {
            'alpha':np.logspace(low,high,num,base=base),
            'normalize':norm,
            'max_iter':[int(max_iter)]
        }
        return param_grid
    
    elif way=='linspace':
        param_grid = {
            'alpha':np.linspace(low,high,num),
            'normalize':norm,
            'max_iter':[int(max_iter)]
        }
        return param_grid

In [ ]:
param = generateLassoParamGrid('logspace',-4,4,100,10,[True],1000)     
param

In [ ]:
search = generateSearch('lasso','grid_search',param,-1,10)     
search

In [ ]:
max_score,best_regr = optHyperparameter(x_data,y_data,test_size,n,search)                      

In [ ]:
max_score

In [ ]:
best_regr

In [ ]:
r2_train,rmse_train,mae_train,r2_test,rmse_test,mae_test,best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,lasso_worst_mae= \
fitRegression(x_data,y_data,test_size,N,best_regr)

In [ ]:
plotBar(r2_train,rmse_train,r2_test,rmse_test,N,'LASSO',label,'saving')

In [ ]:
plotScatter(best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,'LASSO',label,'saving')

In [ ]:
best_lasso = best_regr

In [ ]:
dump(best_lasso, "Models\\lasso_"+label+".joblib")    

In [ ]:
rmse_r2 = [rmse_train, rmse_test, r2_train, r2_test]
mean_rmse_r2 = np.mean(rmse_r2,1) 
std_err = np.std(rmse_r2,axis=1,ddof=1)/np.sqrt(N) 
lasso_predicted_result = [mean_rmse_r2,std_err]
lasso_predicted_result = pd.DataFrame(lasso_predicted_result)
lasso_predicted_result.columns = ["Training rmse", "Testing rmse", "Training R2", "Testing R2"]
lasso_predicted_result.index = ["Mean", "std"]
lasso_predicted_result.to_csv("Excels\\"+"LASSO_"+label+".csv",sep=',')
lasso_mae = [mae_train, mae_test]
lasso_mae = pd.DataFrame(lasso_mae)
lasso_mae.to_csv("Excels\\"+"LASSO_MAE.csv",sep=',')

In [ ]:
end_time_lasso = datetime.datetime.now()
delta_lasso = end_time_lasso - end_time_xgbr
print(end_time_lasso)
print(delta_lasso)

# keras ANNs 

In [ ]:
def generateAnnsParamGrid(low1,high1,num1,low2,high2,num2):      
        param_grid = {
            'batch_size':np.arange(int(low1),int(high1),int(num1)),
            'epochs' :np.arange(int(low2),int(high2),int(num2)),
            'optimizer' : ['adam','sgd']
        }
        return param_grid

In [ ]:
param = generateAnnsParamGrid(10,50,5,100,700,100)  
param

In [ ]:
search = generateSearch('anns','grid_search',param,-1,10)     
search

In [ ]:
max_score,best_regr = optHyperparameter(x_data,y_data,test_size,n,search)  

In [ ]:
max_score

In [ ]:
best_regr

In [ ]:
bs = best_regr.sk_params['batch_size']
es = best_regr.sk_params['epochs']
op = best_regr.sk_params['optimizer']
print('最佳batch_size: %s ，最佳epochs: %s，最佳optimizer: %s'%(bs,es,op))

In [ ]:
r2_train,rmse_train,mae_train,r2_test,rmse_test,mae_test,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,anns_worst_mae= \
fitRegression(x_data,y_data,test_size,N,best_regr)

In [ ]:
plotBar(r2_train,rmse_train,r2_test,rmse_test,N,'ANNs',label,'saving')

In [ ]:
plotScatter(best_r2,best_rmse,y_train_best,y_train_predict_best,y_test_best,y_test_predict_best,'ANNs',label,'saving')

In [ ]:
best_regr.model.save("Models\\anns_"+label+".h5")

In [ ]:
rmse_r2 = [rmse_train, rmse_test, r2_train, r2_test]
mean_rmse_r2 = np.mean(rmse_r2,1) 
std_err = np.std(rmse_r2,axis=1,ddof=1)/np.sqrt(N) 
anns_predicted_result = [mean_rmse_r2,std_err]
anns_predicted_result = pd.DataFrame(anns_predicted_result)
anns_predicted_result.columns = ["Training rmse", "Testing rmse", "Training R2", "Testing R2"]
anns_predicted_result.index = ["Mean", "std"]
anns_predicted_result.to_csv("Time\\"+"ANNS_"+label+".csv",sep=',')
anns_mae = [mae_train, mae_test]
anns_mae = pd.DataFrame(anns_mae)
anns_mae.to_csv("Excels\\"+"ANNS_MAE.csv",sep=',')

In [ ]:
end_time_anns = datetime.datetime.now()
delta_anns = end_time_anns - end_time_lasso
print(end_time_anns)
print(delta_anns)

# The ending of time

In [ ]:
time_comp = [delta_svr,delta_krr,delta_rfr,delta_xgbr,delta_lasso,delta_anns]
time_comp = pd.DataFrame(time_comp)
time_comp.columns=['time cost']
time_comp.index = ['SVR','KRR','RFR','XGBR','LASSO','ANNs']
time_comp.to_csv("Time\\"+"Time_"+label+".csv",sep=',')

# The worst MAE in a trial for best model

In [ ]:
worst_mae = [svr_worst_mae,krr_worst_mae,rfr_worst_mae,xgbrt_worst_mae,lasso_worst_mae,anns_worst_mae] 
worst_mae = pd.DataFrame(worst_mae)
worst_mae.columns=['mae']
worst_mae.index = ['SVR','KRR','RFR','XGBR','LASSO','ANNs']
worst_mae.to_csv("Worst\\"+"worstMAE_"+label+".csv",sep=',')

# Delete global variable

In [ ]:
del N,n,label,test_size